# Select and Register Model from AutoML Runs

## Connect to workspace

In [38]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Experiment

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

Workspace Name: mlonazure-ws
Resource Group: mlonazure-rg
Default Storage Account Name: mlonazurews7669880942
AzureML Core Version: 1.17.0


## Update these variables

In [35]:
autoMLRunId = 'AutoML_938fe675-8c68-433a-8c20-2b2879f69c54'
modelDescription = 'OJ Sales Forecasting Model'
experiment_name = 'CETA_DemandForecasting_Training'
api_service_name = 'forecastdemand-api'
aks_compute_name = 'InferenceCluster'

## Retrieve and Register Model from AutoML Experiment

In [3]:
from azureml.train.automl.run import AutoMLRun

experiment = Experiment(workspace = mlonazure_ws, name = experiment_name)
previous_automl_run = AutoMLRun(experiment, autoMLRunId, outputs = None)
automl_run = previous_automl_run

In [4]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
CETA_DemandForecasting_Training,AutoML_938fe675-8c68-433a-8c20-2b2879f69c54,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
best_run, fitted_model = automl_run.get_output()
print(fitted_model.steps)
model_name = best_run.properties['model_name']
print(model_name)

[('timeseriestransformer', TimeSeriesTransformer(featurization_config=None,
                      pipeline_type=<TimeSeriesPipelineType.FULL: 1>)), ('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('84',
                                          Pipeline(memory=None,
                                                   steps=[('sparsenormalizer',
                                                           <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7fd1a7d3b080>),
                                                          ('xgboostregressor',
                                                           XGBoostRegressor(base_score=0.5,
                                                                            booster='gbtree',
                                                                            colsample_bylevel=0.7,
                                                                            colsample_bynode=1,
                  

In [7]:
model = automl_run.register_model(model_name = model_name, description = modelDescription, tags =  {'Type':'AutoML', 'Data':'iris'})

print(automl_run.model_id)

AutoML938fe6758118


## Deploy Best Model

In [31]:
script_file_name = 'score_fcast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

conda_file_name = 'conda_file.yml'
best_run.download_file('outputs/conda_env_v_1_0_0.yml', conda_file_name)

#automlenv = Environment.from_conda_specification(name = "automlenv", file_path = conda_file_name)


In [32]:
from azureml.core import Environment
envs = Environment.list(mlonazure_ws)
envs.keys()

dict_keys(['mlonazure--env3', 'samsenv', 'AzureML-Tutorial', 'AzureML-Minimal', 'AzureML-Chainer-5.1.0-GPU', 'AzureML-PyTorch-1.2-CPU', 'AzureML-TensorFlow-1.12-CPU', 'AzureML-TensorFlow-1.13-CPU', 'AzureML-PyTorch-1.1-CPU', 'AzureML-TensorFlow-1.10-CPU', 'AzureML-PyTorch-1.0-GPU', 'AzureML-TensorFlow-1.12-GPU', 'AzureML-TensorFlow-1.13-GPU', 'AzureML-Chainer-5.1.0-CPU', 'AzureML-PyTorch-1.0-CPU', 'AzureML-Scikit-learn-0.20.3', 'AzureML-PyTorch-1.2-GPU', 'AzureML-PyTorch-1.1-GPU', 'AzureML-TensorFlow-1.10-GPU', 'AzureML-PyTorch-1.3-GPU', 'AzureML-TensorFlow-2.0-CPU', 'AzureML-PyTorch-1.3-CPU', 'AzureML-TensorFlow-2.0-GPU', 'AzureML-PySpark-MmlSpark-0.15', 'AzureML-AutoML', 'AzureML-PyTorch-1.4-GPU', 'AzureML-PyTorch-1.4-CPU', 'AzureML-VowpalWabbit-8.8.0', 'AzureML-Hyperdrive-ForecastDNN', 'AzureML-AutoML-GPU', 'AzureML-AutoML-DNN-GPU', 'AzureML-AutoML-DNN', 'AzureML-Designer-R', 'AzureML-Designer-Recommender', 'AzureML-Designer-Transform', 'AzureML-Designer', 'AzureML-Designer-IO', 'Az

In [33]:
myenv = Environment(name = "AzureML-AutoML")

In [36]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute, ComputeTarget

aks_target = AksCompute(workspace=mlonazure_ws,name=aks_compute_name)
aks_config = AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,auth_enabled=True)

inference_config = InferenceConfig(environment=myenv,
                                     entry_script = script_file_name)


api_service = Model.deploy(workspace=mlonazure_ws,
                           name=api_service_name, 
                           models=[model],
                           inference_config=inference_config, 
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                          overwrite=True)

api_service.wait_for_deployment(show_output=True)
print(api_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [37]:
pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlonazure-compute-f4s/code/users/mlonazure/AzureMachineLearning/AutoML Forecasting'